In [2]:
import os
import ast
import random
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from sklearn.model_selection import KFold

root = 'data_processed/'

# Generation with Recurrent Neural Networks

To establish a baseline of music generation that we can improve on, we use Recurrent Neural Networks. We formulate the problem as a next-note prediciton problem. This method is quite similar to  recurrence-based language models that are used in NLP.

The input is sequential, but unlike words in NLP, timing and dynamics (duration, velocity, offset) matter a lot in music. To be able to predict notes/chords + durations + offsets + velocities we might need multi-output heads (e.g., softmax for notes/chords/velocities, regression for durations/offsets).

## Import Dataset and Definition of Useful functions

### Import Dataset

In [3]:
def safe_parse_all_columns_df(df):
    """
    Parse all columns in a DataFrame to numeric, coercing errors.
    """
    df['notes'] = df['notes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['chords'] = df['chords'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['velocities'] = df['velocities'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['durations'] = df['durations'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['offsets'] = df['offsets'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df['ordered_events'] = df['ordered_events'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

def load_dataframe_from_two_csvs(file1, file2):
    """
    Load and concatenate two CSV files into a single pandas DataFrame.
    """
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    full_df = pd.concat([df1, df2], ignore_index=True)
    full_df = safe_parse_all_columns_df(full_df)

    return full_df

def save_dataframe_to_two_csvs(df, file1, file2):
    """
    Split a DataFrame in half and save it into two CSV files.
    """
    halfway = len(df) // 2
    df.iloc[:halfway].to_csv(file1, index=False)
    df.iloc[halfway:].to_csv(file2, index=False)

def load_dataframe_from_one_csv(file):
    """
    Load a DataFrame from a single CSV file.
    """
    df = pd.read_csv(file)
    
    return df

def save_dataframe_to_one_csv(df, file):
    """
    Save a DataFrame to a single CSV file.
    """
    df.to_csv(file, index=True)

def load_reconstructed_events(file):
    """
    Loads the reconstructed events CSV and safely parses the 'sequence' column,
    converting notes to integers and chords to lists of integers.
    """
    df = pd.read_csv(file)

    def safe_parse(seq_str):
        try:
            parsed = ast.literal_eval(seq_str)
            if not isinstance(parsed, list):
                raise ValueError("Parsed sequence is not a list")

            normalized = []
            for el in parsed:
                if isinstance(el, list):
                    normalized.append([int(x) for x in el])
                else:
                    normalized.append(int(el))
            return normalized

        except Exception as e:
            print(f"Error parsing sequence: {seq_str}")
            raise e

    df['sequence'] = df['sequence'].apply(safe_parse)
    return df

In [4]:
file1 = root + 'data_part1.csv'
file2 = root + 'data_part2.csv'

df = load_dataframe_from_two_csvs(file1, file2)

### Useful functions

In [5]:
def parse_chord_to_list(chord):
    """
    Convert a chord string to a list of integers.
    """
    if isinstance(chord, str):
        print([int(x) for x in chord.split(',') if x.isdigit()])
        return [int(x) for x in chord.split(',') if x.isdigit()]
    return []

def reconstruct_ordered_events(df):
    """
    Reconstruct the ordered list of events (notes and chords) for each song.
    """
    sequences  = []

    for i in range(len(df)):
        idx_note = 0
        idx_chord = 0
        reconstructed = []

        for element in df['ordered_events'][i]:
            if element == 'n':
                reconstructed.append(df['notes'][i][idx_note])
                idx_note += 1
            elif element == 'c':
                parsed_chord = parse_chord_to_list(df['chords'][i][idx_chord])
                reconstructed.append(df['chords'][i][idx_chord])
                idx_chord += 1
            else:
                raise ValueError(f"Unknown event type: {e}")
        
        sequences.append(reconstructed)

    reconstructed_dataset = pd.DataFrame({'sequence': sequences})
    reconstructed_dataset.index.name = 'index'

    return reconstructed_dataset

In [ ]:
save_dataframe_to_one_csv(reconstruct_ordered_events(df), root + 'reconstructed_ordered_events.csv')

## Predict only Events (Notes and Chords)

### Creating the data: Fixed number of events 

Idea for creating the input sequences:
- we take subsets of the list of events representing each song 
- we take the next event of each subset as corresponding training output sequences

This is easy to implement and we will have a consistent sequence lenght for batching, but we are ignoring the timing aspect.

In [6]:
class Vocabulary:
    def __init__(self, reconstructed_df):
        """
        Build vocabulary of unique single notes only.
        """
        self.notes = set()
        for i in range(len(reconstructed_df)):
            sequence = reconstructed_df['sequence'][i]
            for event in sequence:
                if isinstance(event, list):
                    for note in event:
                        self.notes.add(note)
                else:
                    self.notes.add(event)

        self.notes = sorted(self.notes)
        self.note_to_idx = {note: idx for idx, note in enumerate(self.notes)}
        self.idx_to_note = {idx: note for idx, note in enumerate(self.notes)}
        self.vocab_size = len(self.notes)

    def encode_event(self, event):
        """
        Encode an event as a multi-hot vector over single notes.
        """
        vec = np.zeros(self.vocab_size, dtype=np.float32)
        if isinstance(event, list):
            for note in event:
                vec[self.note_to_idx[note]] = 1.0
        else:
            vec[self.note_to_idx[event]] = 1.0
        return vec

    def decode_event(self, vec, threshold=0.5):
        """
        Decode multi-hot vector to list of notes.
        """
        indices = np.where(vec >= threshold)[0]
        notes = [self.idx_to_note[idx] for idx in indices]
        if len(notes) == 1:
            return notes[0]
        else:
            return notes

    def __len__(self):
        return self.vocab_size


Create Dataset object

In [7]:
class MusicEventDataset(Dataset):
    def __init__(self, reconstructed_df, vocab, seq_length=50):
        """
        Constructs all valid (input_seq, target_event) pairs from each song in the dataset.

        Args:
            reconstructed_df: DataFrame with 'sequence' column where each entry is a list of events
            vocab: Vocabulary object to encode events
            seq_length: Length of each training input sequence (target is the next event)
        """
        self.samples = []
        self.seq_length = seq_length
        self.vocab = vocab

        for row_index in range(len(reconstructed_df)):
            sequence = reconstructed_df['sequence'][row_index]
            n_events = len(sequence)

            if n_events <= seq_length:
                continue

            for i in range(n_events - seq_length):
                input_seq = sequence[i : i + seq_length]
                target_event = sequence[i + seq_length]
                self.samples.append((input_seq, target_event))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        input_seq, target_event = self.samples[idx]

        input_encoded = np.array([self.vocab.encode_event(event) for event in input_seq], dtype=np.float32)
        input_tensor = torch.tensor(input_encoded)

        target_encoded = self.vocab.encode_event(target_event)
        target_tensor = torch.tensor(target_encoded, dtype=torch.float32)

        return input_tensor, target_tensor


In [ ]:
reconstructed_dataset = load_reconstructed_events(root + 'reconstructed_ordered_events.csv')

vocab = Vocabulary(reconstructed_dataset)

dataset = MusicEventDataset(reconstructed_dataset, vocab=vocab, seq_length=16)

x, y = dataset[0]

print("Input sequence shape:", x.shape)
print("Next event shape:", y.shape)
print("Input sequence (multi-hot vectors):", x)
print("Next event (multi-hot vector):", y)

Input sequence shape: torch.Size([16, 88])
Next event shape: torch.Size([88])
Input sequence (multi-hot vectors): tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Next event (multi-hot vector): tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


### Model, Metrics and Cross-validation over sequence length

Model

In [8]:
class MusicEventRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(MusicEventRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                           num_layers=num_layers, batch_first=True)
        
        # Use output_size here explicitly
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden=None):
        out, hidden = self.rnn(x, hidden)  
        out_last = out[:, -1, :]
        out = self.fc(out_last)

        return out, hidden

Cross-validation Loop on Sequence Length

In [9]:
def train_model(model, dataset, epochs=10, batch_size=32, seq_length=16, val_split=0.1, print_every=5, checkpoint_path="best_model.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model = model.to(device)

    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters())

    best_val_loss = float('inf')

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader, 1):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs, _ = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if batch_idx % print_every == 0 or batch_idx == len(train_loader):
                avg_loss = running_loss / batch_idx
                print(f"Epoch {epoch}, Batch {batch_idx}/{len(train_loader)}, Avg Loss: {avg_loss:.4f}")

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                outputs, _ = model(x_val)
                loss = criterion(outputs, y_val)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f"Epoch {epoch} completed, Validation Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Validation loss improved. Model saved to {checkpoint_path}")

    print("Training done.")

In [10]:
reconstructed_dataset = load_reconstructed_events(root + 'reconstructed_ordered_events.csv')
vocab = Vocabulary(reconstructed_dataset)

In [ ]:
# code with hyperparameter tuning

In [ ]:
def evaluate_model(model, val_dataset, batch_size=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    criterion = nn.BCEWithLogitsLoss()
    val_loss = 0.0

    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(device), y_val.to(device)
            outputs, _ = model(x_val)
            loss = criterion(outputs, y_val)
            val_loss += loss.item()

    return val_loss / len(val_loader)


In [ ]:
seq_len = 32
hidden_size = 256
num_layers = 1
batch_size = 128
epochs = 10
learning_rate = 0.001

dataset = MusicEventDataset(reconstructed_dataset, vocab=vocab, seq_length=seq_len)

generator = torch.Generator().manual_seed(42)
total_size = len(dataset)
test_size = int(0.1 * total_size)
train_size = total_size - test_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=generator)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = MusicEventRNN(
    input_size=len(vocab),
    hidden_size=hidden_size,
    output_size=len(vocab),
    num_layers=num_layers
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()

print(f"Training model with seq_len={seq_len}, hidden_size={hidden_size}, for {epochs} epochs...")
for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs, _ = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}/{epochs} - Avg Loss: {avg_loss:.4f}")

torch.save(model.state_dict(), "music_gen_seq32_hidden256.pth")
print("Model saved as 'music_gen_seq32_hidden256.pth'")


Training model with seq_len=32, hidden_size=256, for 10 epochs...


## Predict Events + Durations + Offsets + Velocities

### Creating the data: Fixed time window

The input is sequential, but unlike words in NLP, timing and dynamics (duration, velocity, offset) matter a lot in music. To be able to predict notes/chords + durations + offsets + velocities we might need multi-output heads (e.g., softmax for notes/chords/velocities, regression for durations/offsets).

This method respects the temporal structure of the music, but batching becomes trickier.